In [15]:
groups={
    'dog':{
        'prompt':[
            "a dog is eating a cake", 
            "a dog is eating a strawberry cake", 
            "a Welsh Corgi is eating a cake",
            "a Welshi Corgi is eating a strawberry cake",
            "a brown Welshi Corgi is eating a strawberry cake",
            "a brown Welshi Corgi is eating a delicious strawberry cake"],
        'image_tot':100
    },
    'bear':{
        'prompt':[
            "fish and animal",
            "fish and animals in water",
            "salmon and animals in water",
            "salmon and bears in water",
            "salmon and bears in hot springs",
            "Salmon and bears in hot springs in summer"],
        'image_tot':300
    },
    'food':{
        'prompt':[
            "a dog is eating a cake",
            "a dog is eating a red cake",
            "a dog is eating a strawberry cake",
            "a dog is eating a crepe cake",
            "a dog is eating a cheese cake",
            "a dog is eating a wedding cake",
            "a dog is eating a multi-tier cake",
            "a dog is eating a cup cake"],
        'image_tot':800
    },
    'train':{
        'prompt':[
            "Burning train derailed in the desert",
            "Burning train derailed in the rock desert",
            "Burning Steam locomotive derailed in the desert",
            "Burning Steam locomotive derailed in the rock desert",
            "Burning bkack Steam locomotive derailed in the rock desert",
            "Burning Steam locomotive derailed in the orange rock desert",
            "Burning black Steam locomotive derailed in the orange rock desert"],
        'image_tot':800
    },
    'salmon':{
        'prompt':[
            "a salmon is in the water",
            "a salmon fish is in the water",
            "a salmon fillet is in the water",
            "a salmon fillet is swimming in the water"],
        'image_tot':800
    }
        
}

In [16]:
import csv
import pandas as pd
import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten



nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AEL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AEL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
target='salmon'

dataset = pd.read_csv("space_explore_"+target+".csv")
dataset.head(5)

columns={
    'prompt':[],
    'image_num':[],
    'group_num':[]
}

for idx,g in enumerate(groups[target]['prompt']):
    for j in range(groups[target]['image_tot']):
        columns['prompt'].append(g)
        columns['image_num'].append(j)
        columns['group_num'].append(str(idx))

dataset['prompt']=columns['prompt']
dataset['group_num']=columns['group_num']
dataset['image_num']=columns['image_num']

dataset.head()


,text,prompt,group_num,image_num
0,a salmon leaping out of the water,a salmon is in the water,0,0
1,a salmon leaping out of the water,a salmon is in the water,0,1
2,a fish jumping out of the water,a salmon is in the water,0,2
3,a salmon leaping out of the water,a salmon is in the water,0,3
4,a fish that is standing in the water,a salmon is in the water,0,4


In [18]:
import spacy
import numpy as np
import csv
from sklearn import linear_model
import matplotlib.pyplot as plt

stop_words = set(nltk.corpus.stopwords.words('english'))

nlp = spacy.load('en_core_web_lg')

def our_tokenizer(sentence):
    ps = nltk.stem.PorterStemmer()
    sentence = sentence.replace(',','')
    sentence = sentence.replace('.','')
    words = nltk.word_tokenize(sentence)
    

    temps = []
    for w in words: 
        if w not in stop_words: 
            temps.append(w) 

    res = []
    for stem in temps:
        res.append(ps.stem(stem))

    return res


def embedding_sim(token1, token2):
    avg=0
    for token in token1:
        max=-2
        for t in token2:
            if nlp(token).similarity(nlp(t))>max : max= nlp(token).similarity(nlp(t))
        avg+=max
    avg=float(avg/len(token1))

    return avg

def embedding_vector(token):
    sent=""
    for t in token:
        sent+=" "+t

    return nlp(sent)


vectors=[]
for i,row in dataset.iterrows():
    vectors.append(embedding_vector(our_tokenizer(row["text"])).vector)

In [19]:
from sklearn.manifold import TSNE

dataset["vector"]=vectors

labels = list()
sentences = dataset["vector"]



tsne_model = TSNE(perplexity=25, n_components=2, n_iter=2000, random_state=0)
new_values = tsne_model.fit_transform(np.asarray(vectors))

x, y = list(), list()

for value in new_values:
    x.append(value[0])
    y.append(value[1])

dataset['x']=x
dataset['y']=y

c:\Users\AEL\anaconda3\envs\pyenv37\lib\site-packages\sklearn\manifold\_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
c:\Users\AEL\anaconda3\envs\pyenv37\lib\site-packages\sklearn\manifold\_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [20]:
dataset.tail()

,text,prompt,group_num,image_num,vector,x,y
3195,a fish in the water with a fish in the background,a salmon fillet is swimming in the water,3,795,"[-0.495298, -1.5058, -3.2153199, 0.940036, 1.9...",-35.756924,-10.385710
3196,salmon being cooked in the water,a salmon fillet is swimming in the water,3,796,"[0.403135, -0.12751752, -2.452475, 0.39107502,...",-1.312767,-1.766830
3197,a fish that is swimming in the water,a salmon fillet is swimming in the water,3,797,"[-0.133195, 0.74497503, -2.5576274, -0.7127949...",-5.200360,-53.372108
3198,a piece of salmon on a blue plate,a salmon fillet is swimming in the water,3,798,"[-1.552532, -0.25145596, -1.92178, 1.98894, 1....",70.067429,2.367498
3199,a piece of salmon in a body of water,a salmon fillet is swimming in the water,3,799,"[-0.080571994, -0.25599998, -1.4229, 0.8559, 0...",-17.651743,14.188164


In [21]:
dataset.to_csv("for_visualization_"+target+".csv")

In [51]:
dataset.shape

(6400, 7)